# 06 — Hipótese 2: Augmentation Forte
## ResNet18 | Strong Augmentation | Sem Class Weight

Objetivo:

Testar se aumento na intensidade do data augmentation
melhora a capacidade de generalização do modelo.

Mantemos:
- Arquitetura ResNet18
- Mesmos hiperparâmetros
- Mesmo split congelado

Alteramos apenas:
- augmentation = "strong"

Arquivos utilizados:
- data/metadata/train_split.csv
- data/metadata/val_split.csv

Arquivos gerados:
- models/resnet18_strong_noCW.pt
- outputs/metrics/resnet18_strong_noCW.pkl

In [1]:
import os

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

print("Project root:", PROJECT_ROOT)

Project root: c:\projects\xray-project


In [2]:
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from dataset import XRayDataset
from transforms import get_transforms
from model import get_model
from train_utils import train_model
from utils import set_seed, create_directories, save_metrics
from torch.utils.data import DataLoader

## Controle de Reprodutibilidade

Fixar a mesma seed utilizada nos experimentos anteriores.

In [3]:
set_seed(42)

## Preparação de Diretórios

Garantir que existam:
- models/
- outputs/metrics/

In [4]:
models_dir, metrics_dir, _ = create_directories(PROJECT_ROOT)

print("Models:", models_dir)
print("Metrics:", metrics_dir)

Models: c:\projects\xray-project\models
Metrics: c:\projects\xray-project\outputs\metrics


## Carregamento dos Splits

Utilizar exatamente os mesmos arquivos:
- train_split.csv
- val_split.csv

Isso garante controle experimental.

In [5]:
metadata_dir = os.path.join(PROJECT_ROOT, "data", "metadata")

train_df = pd.read_csv(os.path.join(metadata_dir, "train_split.csv"))
val_df = pd.read_csv(os.path.join(metadata_dir, "val_split.csv"))

print("Treino:", len(train_df))
print("Validação:", len(val_df))

Treino: 4093
Validação: 1139


## Configuração da Hipótese 2

Modelo: ResNet18  
Augmentation: strong  
Class Weight: não utilizado  
Épocas: 10  
Batch Size: 16  
Learning Rate: 1e-4

In [6]:
model_name = "resnet18"
augmentation = "strong"
use_class_weight = False

batch_size = 16
epochs = 10
learning_rate = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)

Dispositivo: cpu


## Datasets e DataLoaders

A única alteração real deste experimento
é o parâmetro augmentation="strong".

In [7]:
train_transform, val_transform = get_transforms(
    img_size=224,
    augmentation=augmentation
)

train_dataset = XRayDataset(train_df, transform=train_transform)
val_dataset = XRayDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Modelo

Arquitetura permanece ResNet18.

In [8]:
model, target_layer = get_model(model_name=model_name)
model = model.to(device)

c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Função de Perda e Otimizador

Ainda não utilizamos class weight nesta etapa.

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Padrão de Nomeação

modelo_augmentation_classweight

In [10]:
model_filename = f"{model_name}_{augmentation}_{'CW' if use_class_weight else 'noCW'}.pt"
metrics_filename = f"{model_name}_{augmentation}_{'CW' if use_class_weight else 'noCW'}.pkl"

model_save_path = os.path.join(models_dir, model_filename)
metrics_save_path = os.path.join(metrics_dir, metrics_filename)


## Treinamento

O loop de treinamento está implementado em src/train_utils.py.

A cada época são exibidos:
- Loss de treino
- Loss de validação
- ROC-AUC de validação
- F1 de validação 
- Recall de validação
- Prec. de validação

O modelo conforme maior AUC é salvo automaticamente.


In [11]:
history, best_auc = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs,
    model_save_path
)


Epoch 1/10
Train Loss: 0.1506
Val Loss:   0.1216
Val AUC:    0.9965
Val F1:     0.9688
Val Recall: 0.9427
Val Prec.:  0.9964
→ Novo melhor modelo salvo.



Epoch 2/10
Train Loss: 0.0983
Val Loss:   0.0565
Val AUC:    0.9973
Val F1:     0.9879
Val Recall: 0.9862
Val Prec.:  0.9896
→ Novo melhor modelo salvo.



Epoch 3/10
Train Loss: 0.0841
Val Loss:   0.0996
Val AUC:    0.9971
Val F1:     0.9797
Val Recall: 0.9954
Val Prec.:  0.9644



Epoch 4/10
Train Loss: 0.0549
Val Loss:   0.1066
Val AUC:    0.9975
Val F1:     0.9726
Val Recall: 0.9966
Val Prec.:  0.9497
→ Novo melhor modelo salvo.



Epoch 5/10
Train Loss: 0.0656
Val Loss:   0.0391
Val AUC:    0.9988
Val F1:     0.9908
Val Recall: 0.9862
Val Prec.:  0.9954
→ Novo melhor modelo salvo.



Epoch 6/10
Train Loss: 0.0581
Val Loss:   0.0463
Val AUC:    0.9988
Val F1:     0.9885
Val Recall: 0.9828
Val Prec.:  0.9942



Epoch 7/10
Train Loss: 0.0486
Val Loss:   0.1248
Val AUC:    0.9978
Val F1:     0.9718
Val Recall: 0.9472
Val Prec.:  0.9976



Epoch 8/10
Train Loss: 0.0397
Val Loss:   0.0508
Val AUC:    0.9985
Val F1:     0.9873
Val Recall: 0.9817
Val Prec.:  0.9930



Epoch 9/10
Train Loss: 0.0383
Val Loss:   0.0444
Val AUC:    0.9987
Val F1:     0.9873
Val Recall: 0.9805
Val Prec.:  0.9942



Epoch 10/10
Train Loss: 0.0394
Val Loss:   0.0594
Val AUC:    0.9990
Val F1:     0.9849
Val Recall: 0.9725
Val Prec.:  0.9976
→ Novo melhor modelo salvo.

Melhor AUC obtida: 0.9989519980757997


## Salvamento das Métricas

As métricas completas serão utilizadas
no notebook final de comparação entre hipóteses.

In [12]:
metrics_dict = {
    "model_name": model_name,
    "augmentation": augmentation,
    "class_weight": use_class_weight,
    "epochs": epochs,
    "history": history,
    "best_auc": best_auc
}

save_metrics(metrics_dict, metrics_save_path)

print("Modelo salvo em:", model_save_path)
print("Métricas salvas em:", metrics_save_path)
print("Melhor AUC:", best_auc)

Modelo salvo em: c:\projects\xray-project\models\resnet18_strong_noCW.pt
Métricas salvas em: c:\projects\xray-project\outputs\metrics\resnet18_strong_noCW.pkl
Melhor AUC: 0.9989519980757997
